# Calculate emissions for IHS materials contained in process recipes using conversion factors

In [1]:
# Import packages
import numpy as np
import pandas as pd
#from tqdm import tqdm

pd.set_option('max_columns', None)
pd.options.mode.chained_assignment = None

OptionError: 'Pattern matched multiple keys'

## Data imports

In [2]:
# Data file paths
input_path = '/Users/lukecullen/Library/CloudStorage/OneDrive-SharedLibraries-UniversityofCambridge/Fanran Meng - chemical emission model (shared)/data/'
output_path = input_path+'/combined/'
ihs_materials_path = input_path+'processed/ihsMaterials_w_uncertainties.csv'
ecoinvent_file = input_path+'processed/conversionFactors_ecoinvent_grouped.csv'
carbonMinds_file = input_path+'processed/conversionFactors_carbonMinds_grouped.csv'
match_list_path = input_path+'extra_inputs/nameMatches_IHS_to_convFactors.csv'

In [3]:
# Import IHS process recipes
ihs_materials = pd.read_csv(ihs_materials_path, index_col=0)
ihs_materials.head()

,Code,Data Version,Source/Object,Type,Target/Process,Research Year,Country/Reg,Product,Value,Value unit,Capacity unit,MeasType,Provenance,Value_sigma
0,2M-1,2021 Q3,ACRYLONITRILE,Raw Material,ABS RESIN BY EMULSION/MASS POLYMERIZATION,1980.0,Germany,ABS RESIN,0.250790,kg/kg,MM KG/yr,Recipe,IHS PEP,0.012540
1,2M-1,2021 Q3,BUTADIENE,Raw Material,ABS RESIN BY EMULSION/MASS POLYMERIZATION,1980.0,Germany,ABS RESIN,0.196470,kg/kg,MM KG/yr,Recipe,IHS PEP,0.009824
2,2M-1,2021 Q3,MISC CHEMICALS,Raw Material,ABS RESIN BY EMULSION/MASS POLYMERIZATION,1980.0,Germany,ABS RESIN,NaN,NaN,MM KG/yr,Recipe,IHS PEP,NaN
3,2M-1,2021 Q3,STYRENE,Raw Material,ABS RESIN BY EMULSION/MASS POLYMERIZATION,1980.0,Germany,ABS RESIN,0.574110,kg/kg,MM KG/yr,Recipe,IHS PEP,0.028705
4,2M-1,2021 Q3,COOLING WATER,Utilities,ABS RESIN BY EMULSION/MASS POLYMERIZATION,1980.0,Germany,ABS RESIN,30.460712,kg/kg,MM KG/yr,Utilities,IHS PEP,1.523036


In [4]:
# Import conversion factors
ei_emissions = pd.read_csv(ecoinvent_file, index_col=0)
cm_emissions = pd.read_csv(carbonMinds_file, index_col=0)

ei_emissions.head()

,Source,generalComment,location,CO2e_20a,CO2e_100a,Carbon dioxide,Carbon monoxide,Chloroform,Dinitrogen monoxide,Ethane,...,Carbon monoxide_sigma,Chloroform_sigma,Dinitrogen monoxide_sigma,Ethane_sigma,Methane_sigma,Nitric oxide_sigma,Nitrogen fluoride_sigma,Perfluoropentane_sigma,Sulfur hexafluoride_sigma,Other_sigma
0,"1,1-difluoroethane, HFC-152a",This is a market activity. Each market represe...,GLO,11.574029,6.023462,3.781133,0.004604,1.811922e-08,0.000138,1.358312e-02,...,0.000460,1.811922e-09,0.000014,1.358312e-03,0.000972,1.662799e-12,1.453954e-17,0.0,1.209512e-07,5.056085e-10
1,"1,1-dimethylcyclopentane",This is a market activity. Each market represe...,GLO,3.194169,2.486416,1.911578,0.003376,3.563351e-09,0.000158,3.670008e-04,...,0.000338,3.563351e-10,0.000016,3.670008e-05,0.001244,3.937246e-12,3.166334e-18,0.0,7.058694e-09,3.251406e-11
2,1-butanol,This is a market activity. Each market represe...,GLO,3.919208,3.150644,2.703243,0.013396,6.852872e-09,0.000040,4.697008e-08,...,0.001340,6.852872e-10,0.000004,4.697008e-09,0.001276,8.137414e-12,2.152075e-18,0.0,1.749540e-08,5.540938e-11
3,1-pentanol,This is a market activity. Each market represe...,GLO,7.308616,6.136045,5.467968,0.008194,8.482948e-09,0.000082,1.058417e-07,...,0.000819,8.482948e-10,0.000008,1.058417e-08,0.002060,9.660171e-12,6.786576e-18,0.0,2.119839e-08,5.023599e-11
4,1-propanol,This is a market activity. Each market represe...,GLO,5.293213,4.502282,4.036386,0.007359,1.348648e-08,0.000080,9.988269e-08,...,0.000736,1.348648e-09,0.000008,9.988269e-09,0.001375,1.260825e-11,9.772094e-18,0.0,3.618003e-08,9.169649e-11


In [10]:
## Add crude oil in g/MJ and 41.686MJ/kg crude
crude_input = pd.DataFrame(np.array([['crude oil input'], ['Sourced from Crude oil Europe'], ['GLO'], [0.012*41.868], [0.012*41.868], [0.012*41.868], [0.005*41.868], [0.005*41.868], [0.005*41.868]]).transpose(), columns=['Source', 'generalComment', 'location', 'CO2e_20a', 'CO2e_100a', 'Carbon dioxide', 'CO2e_20a_sigma', 'CO2e_100a_sigma', 'Carbon dioxide_sigma'])
ei_emissions = pd.concat((ei_emissions, crude_input))

ei_emissions[ei_emissions.columns[3:]] = ei_emissions[ei_emissions.columns[3:]].astype(float)

## Assign emissions from feedstocks and indirect utilities

In [14]:
def filter_rows(df:pd.DataFrame, column:str, item:str, exact:bool=True):
    """Function for finding best match for input item in a df column"""
    # If exact match enforced
    if exact:
        return df[df[column].str.lower() == item.lower()]

    # If item is in string but not entire string
    else: return df[[item in row for row in df[column].str.lower()]]

def uncertainty_propagation(calc:str, x:float, dx:float, y:float=1, dy:float=0, z:float=1, propagation_type:str='simple') -> float:
    """Function for propagating uncertainty through calculations"""
    # Multiplication
    if calc == 'mult':
        xdiv = np.divide(dx, x, out=np.zeros_like(dx), where=x!=0)
        ydiv = np.divide(dy, y, out=np.zeros_like(dy), where=y!=0)
        if propagation_type == 'simple':
            return (xdiv + ydiv)*z
        elif propagation_type == 'stdev':
            return np.sqrt(pow(xdiv,2) + pow(ydiv,2))*z
        else: Exception('Specified propagation_type not recognised.')

    # Addition
    elif calc == 'add':
        if propagation_type == 'simple':
            return abs(dx)+abs(dy)
        elif propagation_type == 'stdev':
            return np.sqrt(pow(dx,2) + pow(dy,2))
        else: Exception('Specified propagation_type not recognised.')
    else: Exception('Please specify calc of propagation')

In [39]:
def assign_emissions(df:pd.DataFrame, emissions_df:pd.DataFrame, product_col:str, emissions_col:str,
                     product_val_col:str='Value', emission_val_cols:list=None, emission_val_cols_sigma:list= None, match_list=None, db_name:str='db', production_unit_conv:float=1, keep_all:bool=False) -> (pd.DataFrame, pd.DataFrame):
    """These function assigns appropriate emissions values from EcoInvent or Carbonminds to products or materials in IHS given a pre-determined match from file or finding the best matches available"""

    # Create values if none exist
    if match_list is None:
        match_list = {}
    if emission_val_cols is None:
        emission_val_cols = ['Cradle-to-gate']
    if emission_val_cols_sigma is None:
        emission_val_cols_sigma = ['Cradle-to-gate_sigma']
    product_val_col_sigma = product_val_col+'_sigma'

    # Create columns to receive emissions, match name, emissions conversion factor
    val_col, match_name_col, conv_factor_col = pd.DataFrame(columns=emission_val_cols), [], pd.DataFrame(columns=emission_val_cols)
    # Columns for uncertainties of above
    val_col_sigma, conv_factor_col_sigma = pd.DataFrame(columns=emission_val_cols_sigma), pd.DataFrame(columns=emission_val_cols_sigma)

    # Create match dictionary from appropriate match dataframe column
    length = len(emission_val_cols+emission_val_cols_sigma)
    if isinstance(match_list, pd.DataFrame) and db_name in match_list.columns:
        match_list = dict(zip(match_list['IHS'], match_list[db_name]))

    # Loop through rows in assignment dataframe
    for row_num, row in tqdm(enumerate(df.iloc())):

        # Check match_list for match
        if row[product_col].lower() in match_list.keys():

            # If already defined as no match in db
            if str(match_list[row[product_col].lower()]) == '0':
                correspondence = pd.DataFrame()
                emission_val, name = pd.DataFrame(np.array([np.NAN]*length).reshape(1,length), columns=emission_val_cols+emission_val_cols_sigma), np.NAN
            # If match has corresponding db value
            else:
                correspondence = filter_rows(emissions_df, emissions_col, match_list[row[product_col].lower()])
                emission_val = correspondence[emission_val_cols+emission_val_cols_sigma]
                name = correspondence.iloc[0][emissions_col]

        # If no match yet assigned
        else:
            # Find correspondence in emissions dataframe
            correspondence = filter_rows(emissions_df, emissions_col, row[product_col].lower()) # Exact matching
            
            if len(correspondence) == 0: # No exact match -> Try name contained within a match
                correspondence = filter_rows(emissions_df, emissions_col, row[product_col].lower(), exact=False)

                if len(correspondence) > 1: # If multiple inexact matches
                        take = input('Enter number of best match for '+row[product_col].lower()+':\n'+str(correspondence[emissions_col])+'\n Type n to skip') # Ask user for best match
                        if take != 'n': 
                            correspondence = correspondence[correspondence[emissions_col]==correspondence.loc[int(take)][emissions_col]] # Take best match
                        else:
                            correspondence = pd.DataFrame() # If none correspond then empty correspondence

            if len(correspondence) == 0: # No exact match -> Try match contained within name
                matching = emissions_df[[i in row[product_col].lower() for i in emissions_df[emissions_col]]] # Emission string contained in row matching

                if len(matching) > 0: # If multiple matches
                    correspondence = matching.iloc[np.argmax([len(i) for i in matching[emissions_col]])] # Take greatest length of match if multiple
                    emission_val = correspondence[emission_val_cols+emission_val_cols_sigma]
                    name = correspondence[emissions_col]

                else: emission_val, name = pd.DataFrame(np.array([np.NAN]*length).reshape(1,length), columns=emission_val_cols+emission_val_cols_sigma), np.NAN # If no matches identified

            else:
                emission_val = correspondence[emission_val_cols+emission_val_cols_sigma]
                name = correspondence[emissions_col].values[0]

            # Add match to match_list
            if len(correspondence) != 0:
                if isinstance(correspondence, pd.DataFrame):
                    match_list.update({row[product_col].lower():correspondence.iloc[0]['Source']})
                else:
                    match_list.update({row[product_col].lower():correspondence['Source']})
            else: match_list.update({row[product_col].lower():0})
            del correspondence

        # Add matching values to dataframe
        val_col = pd.concat((val_col, row[product_val_col]*production_unit_conv*emission_val[emission_val_cols]))

        # Calculate implied uncertainties and add to dataframe
        val_col_sigma = pd.concat((val_col_sigma, uncertainty_propagation('mult', row[product_val_col], row[product_val_col_sigma], emission_val[emission_val_cols], emission_val[emission_val_cols_sigma], z=(row[product_val_col]*production_unit_conv*emission_val[emission_val_cols]).values)*production_unit_conv))

        # Add other parameters to parameter lists
        match_name_col += [name]
        conv_factor_col = pd.concat((conv_factor_col, emission_val[emission_val_cols]))
        conv_factor_col_sigma = pd.concat((conv_factor_col_sigma, emission_val[emission_val_cols_sigma]))

    df[db_name + '_match'] = match_name_col
    for column, sigma_col in zip(emission_val_cols, emission_val_cols_sigma):
        df[db_name + '_' + column + '_cradle-to-gate'] = val_col[column].values
        df[db_name + '_' + column + '_cradle-to-gate_sigma'] = val_col_sigma[sigma_col].values
        df[db_name + '_' + column + '_conv_factor'] = conv_factor_col[column].values
        df[db_name + '_' + column + '_conv_factor_sigma'] = conv_factor_col_sigma[sigma_col].values

    return df, pd.DataFrame.from_dict(match_list, orient='index').reset_index().rename(columns={'index':'IHS', 0:db_name})

In [41]:
# Match equivalent emissions to materials
keep_all = False
if keep_all:
    emission_val_cols = list(ei_emissions.columns[3:5])
    emission_val_cols_sigma = list(ei_emissions.columns[16:18])
else:
    emission_val_cols = list(ei_emissions.columns[3:16])
    emission_val_cols_sigma = list(ei_emissions.columns[16:])

match_list_ei = pd.read_csv(match_list_path, index_col=False, usecols=['IHS','ei'])

# EI matching
material_emissions, upt_list = assign_emissions(ihs_materials.copy(), ei_emissions, 'Source/Object', 'Source', match_list=match_list_ei, db_name='ei', emission_val_cols=emission_val_cols, emission_val_cols_sigma=emission_val_cols_sigma, keep_all=keep_all)

match_list_ei = pd.concat((match_list_ei, upt_list)).drop_duplicates(subset=['IHS'], keep='last')

CM matching
match_list_cm = pd.read_csv(match_list_path, index_col=False, usecols=['IHS','cm'])
material_emissions, upt_list = assign_emissions(material_emissions, cm_emissions, 'Source/Object', 'Source', match_list=match_list_cm, db_name='cm', emission_val_cols=emission_val_cols, emission_val_cols_sigma=emission_val_cols_sigma)
match_list_cm = pd.concat((match_list_cm, upt_list)).drop_duplicates(subset=['IHS'], keep='last')

# Combine match lists
all_matches = match_list_ei[['IHS','ei']]
all_matches['cm'] = match_list_cm['cm']
all_matches.sort_values('IHS').reset_index(drop=True).to_csv(match_list_path, index=False)

# Create materials emissions
material_emissions = material_emissions.drop_duplicates(subset=['Code', 'Source/Object']).reset_index(drop=True)

material_emissions.head()

,Code,Data Version,Source/Object,Type,Target/Process,Research Year,Country/Reg,Product,Value,Value unit,Capacity unit,MeasType,Provenance,Value_sigma,ei_match,ei_CO2e_20a_cradle-to-gate,ei_CO2e_20a_cradle-to-gate_sigma,ei_CO2e_20a_conv_factor,ei_CO2e_20a_conv_factor_sigma,ei_CO2e_100a_cradle-to-gate,ei_CO2e_100a_cradle-to-gate_sigma,ei_CO2e_100a_conv_factor,ei_CO2e_100a_conv_factor_sigma,ei_Carbon dioxide_cradle-to-gate,ei_Carbon dioxide_cradle-to-gate_sigma,ei_Carbon dioxide_conv_factor,ei_Carbon dioxide_conv_factor_sigma,ei_Carbon monoxide_cradle-to-gate,ei_Carbon monoxide_cradle-to-gate_sigma,ei_Carbon monoxide_conv_factor,ei_Carbon monoxide_conv_factor_sigma,ei_Chloroform_cradle-to-gate,ei_Chloroform_cradle-to-gate_sigma,ei_Chloroform_conv_factor,ei_Chloroform_conv_factor_sigma,ei_Dinitrogen monoxide_cradle-to-gate,ei_Dinitrogen monoxide_cradle-to-gate_sigma,ei_Dinitrogen monoxide_conv_factor,ei_Dinitrogen monoxide_conv_factor_sigma,ei_Ethane_cradle-to-gate,ei_Ethane_cradle-to-gate_sigma,ei_Ethane_conv_factor,ei_Ethane_conv_factor_sigma,ei_Methane_cradle-to-gate,ei_Methane_cradle-to-gate_sigma,ei_Methane_conv_factor,ei_Methane_conv_factor_sigma,ei_Nitric oxide_cradle-to-gate,ei_Nitric oxide_cradle-to-gate_sigma,ei_Nitric oxide_conv_factor,ei_Nitric oxide_conv_factor_sigma,ei_Nitrogen fluoride_cradle-to-gate,ei_Nitrogen fluoride_cradle-to-gate_sigma,ei_Nitrogen fluoride_conv_factor,ei_Nitrogen fluoride_conv_factor_sigma,ei_Perfluoropentane_cradle-to-gate,ei_Perfluoropentane_cradle-to-gate_sigma,ei_Perfluoropentane_conv_factor,ei_Perfluoropentane_conv_factor_sigma,ei_Sulfur hexafluoride_cradle-to-gate,ei_Sulfur hexafluoride_cradle-to-gate_sigma,ei_Sulfur hexafluoride_conv_factor,ei_Sulfur hexafluoride_conv_factor_sigma,ei_Other_cradle-to-gate,ei_Other_cradle-to-gate_sigma,ei_Other_conv_factor,ei_Other_conv_factor_sigma,cm_match,cm_CO2e_20a_cradle-to-gate,cm_CO2e_20a_cradle-to-gate_sigma,cm_CO2e_20a_conv_factor,cm_CO2e_20a_conv_factor_sigma,cm_CO2e_100a_cradle-to-gate,cm_CO2e_100a_cradle-to-gate_sigma,cm_CO2e_100a_conv_factor,cm_CO2e_100a_conv_factor_sigma,cm_Carbon dioxide_cradle-to-gate,cm_Carbon dioxide_cradle-to-gate_sigma,cm_Carbon dioxide_conv_factor,cm_Carbon dioxide_conv_factor_sigma,cm_Carbon monoxide_cradle-to-gate,cm_Carbon monoxide_cradle-to-gate_sigma,cm_Carbon monoxide_conv_factor,cm_Carbon monoxide_conv_factor_sigma,cm_Chloroform_cradle-to-gate,cm_Chloroform_cradle-to-gate_sigma,cm_Chloroform_conv_factor,cm_Chloroform_conv_factor_sigma,cm_Dinitrogen monoxide_cradle-to-gate,cm_Dinitrogen monoxide_cradle-to-gate_sigma,cm_Dinitrogen monoxide_conv_factor,cm_Dinitrogen monoxide_conv_factor_sigma,cm_Ethane_cradle-to-gate,cm_Ethane_cradle-to-gate_sigma,cm_Ethane_conv_factor,cm_Ethane_conv_factor_sigma,cm_Methane_cradle-to-gate,cm_Methane_cradle-to-gate_sigma,cm_Methane_conv_factor,cm_Methane_conv_factor_sigma,cm_Nitric oxide_cradle-to-gate,cm_Nitric oxide_cradle-to-gate_sigma,cm_Nitric oxide_conv_factor,cm_Nitric oxide_conv_factor_sigma,cm_Nitrogen fluoride_cradle-to-gate,cm_Nitrogen fluoride_cradle-to-gate_sigma,cm_Nitrogen fluoride_conv_factor,cm_Nitrogen fluoride_conv_factor_sigma,cm_Perfluoropentane_cradle-to-gate,cm_Perfluoropentane_cradle-to-gate_sigma,cm_Perfluoropentane_conv_factor,cm_Perfluoropentane_conv_factor_sigma,cm_Sulfur hexafluoride_cradle-to-gate,cm_Sulfur hexafluoride_cradle-to-gate_sigma,cm_Sulfur hexafluoride_conv_factor,cm_Sulfur hexafluoride_conv_factor_sigma,cm_Other_cradle-to-gate,cm_Other_cradle-to-gate_sigma,cm_Other_conv_factor,cm_Other_conv_factor_sigma
0,2M-1,2021 Q3,ACRYLONITRILE,Raw Material,ABS RESIN BY EMULSION/MASS POLYMERIZATION,1980.0,Germany,ABS RESIN,0.250790,kg/kg,MM KG/yr,Recipe,IHS PEP,0.012540,acrylonitrile,0.879914,0.131987,3.508568,0.350857,0.847752,0.127163,3.380325,0.338033,0.827089,0.124063,3.297936,0.329794,0.001193,0.000179,0.004757,0.000476,3.466419e-09,5.199629e-10,1.382200e-08,1.382200e-09,4.673512e-06,7.010268e-07,1.863516e-05,1.863516e-06,1.303188e-08,1.954782e-09

In [42]:
# Output emissions per material for process recipes
material_emissions.to_csv(output_path+'ihsMaterialsEmissions_w_upstream.csv')

## Assign emissions from direct utilities

In [43]:
# Read in material emissions (with feedstock & indirect utilities)
material_emissions = pd.read_csv(output_path+'ihsMaterialsEmissions_w_upstream.csv', index_col=0)
direct_utl_conv = pd.read_csv(input_path+'extra_inputs/direct_utility_conversion_factors.csv')

material_emissions['Type'] = material_emissions['Type'].replace({'Utilities':'Indirect Utilities'})

In [44]:
# Add direct emissions for each utility
emission_val_cols = list(ei_emissions.columns[3:16])
emission_val_cols_sigma = list(ei_emissions.columns[16:])

direct_utl_ems = material_emissions[material_emissions['Type']=='Indirect Utilities'][material_emissions.columns[:14]]
direct_utl_ems['Type'] = 'Direct Utilities'
direct_utils = direct_utl_ems.merge(direct_utl_conv, left_on='Source/Object', right_on='Source', how='left').rename(columns={'Source':'ei_match'})

for col in emission_val_cols+emission_val_cols_sigma+['Value', 'Value_sigma']:
    direct_utils[col] = direct_utils[col].astype(float)

for gas in emission_val_cols:
    direct_utils['ei_'+gas+'_cradle-to-gate'] = direct_utils['Value']*direct_utils[gas]
    direct_utils['ei_'+gas+'_cradle-to-gate_sigma'] = uncertainty_propagation('mult', direct_utils['Value'], direct_utils['Value_sigma'], direct_utils[gas], direct_utils[gas+'_sigma'], z=direct_utils['ei_'+gas+'_cradle-to-gate'])
    direct_utils['ei_'+gas+'_conv_factor'] = direct_utils[gas]
    direct_utils['ei_'+gas+'_conv_factor_sigma'] = direct_utils[gas+'_sigma']

direct_utils.drop(columns=emission_val_cols+emission_val_cols_sigma, inplace=True)

In [45]:
# Merge with material emissions
input_emissions = pd.concat((material_emissions, direct_utils), axis='index').sort_values(by=['Product', 'Target/Process', 'Code', 'Type', 'Source/Object'])

input_emissions.to_csv(output_path+'ihsMaterialsEmissions_w_utilities.csv')

## Assign emissions from direct process

In [5]:
# Import materials from previous and define emission_val_cols

emission_val_cols = list(ei_emissions.columns[3:16])
emission_val_cols_sigma = list(ei_emissions.columns[16:])

input_emissions = pd.read_csv(output_path+'ihsMaterialsEmissions_w_utilities.csv', index_col=0)
input_emissions.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lukecullen/Library/CloudStorage/OneDrive-SharedLibraries-UniversityofCambridge/Fanran Meng - chemical emission model (shared)/data//combined/ihsMaterialsEmissions_w_utilities.csv'

In [6]:
pd.read_excel(input_path+'extra_inputs/Direct process emissions.xlsx', skiprows=2)[1:].dropna(subset=['Process']).sort_values('Process').reset_index(drop=True)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [47]:
# Import direct emissions and match to existing products in ihsMaterials

direct_emissions = pd.read_excel(input_path+'extra_inputs/Direct process emissions.xlsx', skiprows=2)[1:].dropna(subset=['Process']).sort_values('Process').reset_index(drop=True)
direct_emissions = direct_emissions[['Process']+list(direct_emissions.columns[-5:])]
direct_emissions['Process'] = direct_emissions['Process'].str.upper()

product_process_match = pd.read_csv(input_path+'extra_inputs/product_to_directProcess_matches.csv')

direct_emissions = direct_emissions.merge(product_process_match, on='Process', how='right').dropna(subset=['Product']).drop(columns=['Process']).drop_duplicates(subset=['Product']).rename(columns={'est. CO2':'Carbon dioxide', 'est. CH4':'Methane','est. N2O':'Nitric oxide', 'est. CO2e_20a':'CO2e_20a', 'est. CO2e_100a': 'CO2e_100a'})

uncertainty_ratio = 0.01

for col in emission_val_cols:
    if col in list(direct_emissions.columns):
        direct_emissions['ei_'+col+'_cradle-to-gate'] = direct_emissions[col].fillna(0).astype(float)
        direct_emissions['ei_'+col+'_cradle-to-gate_sigma'] = (direct_emissions[col].astype(float)*uncertainty_ratio)
        direct_emissions.drop(columns=[col], inplace=True)
    else:
        direct_emissions['ei_'+col+'_cradle-to-gate'] = 0
        direct_emissions['ei_'+col+'_cradle-to-gate_sigma'] = 0

direct_emissions.head()

,Product,ei_CO2e_20a_cradle-to-gate,ei_CO2e_20a_cradle-to-gate_sigma,ei_CO2e_100a_cradle-to-gate,ei_CO2e_100a_cradle-to-gate_sigma,ei_Carbon dioxide_cradle-to-gate,ei_Carbon dioxide_cradle-to-gate_sigma,ei_Carbon monoxide_cradle-to-gate,ei_Carbon monoxide_cradle-to-gate_sigma,ei_Chloroform_cradle-to-gate,ei_Chloroform_cradle-to-gate_sigma,ei_Dinitrogen monoxide_cradle-to-gate,ei_Dinitrogen monoxide_cradle-to-gate_sigma,ei_Ethane_cradle-to-gate,ei_Ethane_cradle-to-gate_sigma,ei_Methane_cradle-to-gate,ei_Methane_cradle-to-gate_sigma,ei_Nitric oxide_cradle-to-gate,ei_Nitric oxide_cradle-to-gate_sigma,ei_Nitrogen fluoride_cradle-to-gate,ei_Nitrogen fluoride_cradle-to-gate_sigma,ei_Perfluoropentane_cradle-to-gate,ei_Perfluoropentane_cradle-to-gate_sigma,ei_Sulfur hexafluoride_cradle-to-gate,ei_Sulfur hexafluoride_cradle-to-gate_sigma,ei_Other_cradle-to-gate,ei_Other_cradle-to-gate_sigma
0,2-ETHYLHEXANOL,-4.068353e-16,-4.068353e-18,-4.068353e-16,-4.068353e-18,-4.068353e-16,-4.068353e-18,0,0,0,0,0,0,0,0,0.00000,0.000000,0.0,0.0,0,0,0,0,0,0,0,0
1,ACETIC ACID,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0.00000,0.000000,0.0,0.0,0,0,0,0,0,0,0,0
2,ACETONE,1.627341e-15,1.627341e-17,1.627341e-15,1.627341e-17,1.627341e-15,1.627341e-17,0,0,0,0,0,0,0,0,0.00000,0.000000,0.0,0.0,0,0,0,0,0,0,0,0
3,PHENOL,1.627341e-15,1.627341e-17,1.627341e-15,1.627341e-17,1.627341e-15,1.627341e-17,0,0,0,0,0,0,0,0,0.00000,0.000000,0.0,0.0,0,0,0,0,0,0,0,0
4,ACRYLONITRILE,1.522800e-02,1.522800e-04,5.346000e-03,5.346000e-05,4.068353e-16,4.068353e-18,0,0,0,0,0,0,0,0,0.00018,0.000002,0.0,0.0,0,0,0,0,0,0,0,0


In [48]:
# Add emissions for each direct process
process_emissions = input_emissions[[i in list(direct_emissions['Product']) for i in list(input_emissions['Product'])]][input_emissions.columns[:14]].drop_duplicates(subset=['Code','Target/Process','Product']).reset_index(drop=True)
process_emissions['Type'], process_emissions['MeasType'] = 'Direct Process', 'Chemical'
process_emissions['Source/Object'] = process_emissions['Product']
process_emissions['Value'], process_emissions['Value_sigma'] = 1, 0
process_emissions = process_emissions.merge(direct_emissions, on='Product', how='inner')

# Merge with all input emissions
output_emissions = pd.concat((input_emissions, process_emissions), axis='index').sort_values(by=['Product', 'Target/Process', 'Code', 'Type', 'Source/Object'])
output_emissions.head()

,Code,Data Version,Source/Object,Type,Target/Process,Research Year,Country/Reg,Product,Value,Value unit,Capacity unit,MeasType,Provenance,Value_sigma,ei_match,ei_CO2e_20a_cradle-to-gate,ei_CO2e_20a_cradle-to-gate_sigma,ei_CO2e_20a_conv_factor,ei_CO2e_20a_conv_factor_sigma,ei_CO2e_100a_cradle-to-gate,ei_CO2e_100a_cradle-to-gate_sigma,ei_CO2e_100a_conv_factor,ei_CO2e_100a_conv_factor_sigma,ei_Carbon dioxide_cradle-to-gate,ei_Carbon dioxide_cradle-to-gate_sigma,ei_Carbon dioxide_conv_factor,ei_Carbon dioxide_conv_factor_sigma,ei_Carbon monoxide_cradle-to-gate,ei_Carbon monoxide_cradle-to-gate_sigma,ei_Carbon monoxide_conv_factor,ei_Carbon monoxide_conv_factor_sigma,ei_Chloroform_cradle-to-gate,ei_Chloroform_cradle-to-gate_sigma,ei_Chloroform_conv_factor,ei_Chloroform_conv_factor_sigma,ei_Dinitrogen monoxide_cradle-to-gate,ei_Dinitrogen monoxide_cradle-to-gate_sigma,ei_Dinitrogen monoxide_conv_factor,ei_Dinitrogen monoxide_conv_factor_sigma,ei_Ethane_cradle-to-gate,ei_Ethane_cradle-to-gate_sigma,ei_Ethane_conv_factor,ei_Ethane_conv_factor_sigma,ei_Methane_cradle-to-gate,ei_Methane_cradle-to-gate_sigma,ei_Methane_conv_factor,ei_Methane_conv_factor_sigma,ei_Nitric oxide_cradle-to-gate,ei_Nitric oxide_cradle-to-gate_sigma,ei_Nitric oxide_conv_factor,ei_Nitric oxide_conv_factor_sigma,ei_Nitrogen fluoride_cradle-to-gate,ei_Nitrogen fluoride_cradle-to-gate_sigma,ei_Nitrogen fluoride_conv_factor,ei_Nitrogen fluoride_conv_factor_sigma,ei_Perfluoropentane_cradle-to-gate,ei_Perfluoropentane_cradle-to-gate_sigma,ei_Perfluoropentane_conv_factor,ei_Perfluoropentane_conv_factor_sigma,ei_Sulfur hexafluoride_cradle-to-gate,ei_Sulfur hexafluoride_cradle-to-gate_sigma,ei_Sulfur hexafluoride_conv_factor,ei_Sulfur hexafluoride_conv_factor_sigma,ei_Other_cradle-to-gate,ei_Other_cradle-to-gate_sigma,ei_Other_conv_factor,ei_Other_conv_factor_sigma,cm_match,cm_CO2e_20a_cradle-to-gate,cm_CO2e_20a_cradle-to-gate_sigma,cm_CO2e_20a_conv_factor,cm_CO2e_20a_conv_factor_sigma,cm_CO2e_100a_cradle-to-gate,cm_CO2e_100a_cradle-to-gate_sigma,cm_CO2e_100a_conv_factor,cm_CO2e_100a_conv_factor_sigma,cm_Carbon dioxide_cradle-to-gate,cm_Carbon dioxide_cradle-to-gate_sigma,cm_Carbon dioxide_conv_factor,cm_Carbon dioxide_conv_factor_sigma,cm_Carbon monoxide_cradle-to-gate,cm_Carbon monoxide_cradle-to-gate_sigma,cm_Carbon monoxide_conv_factor,cm_Carbon monoxide_conv_factor_sigma,cm_Chloroform_cradle-to-gate,cm_Chloroform_cradle-to-gate_sigma,cm_Chloroform_conv_factor,cm_Chloroform_conv_factor_sigma,cm_Dinitrogen monoxide_cradle-to-gate,cm_Dinitrogen monoxide_cradle-to-gate_sigma,cm_Dinitrogen monoxide_conv_factor,cm_Dinitrogen monoxide_conv_factor_sigma,cm_Ethane_cradle-to-gate,cm_Ethane_cradle-to-gate_sigma,cm_Ethane_conv_factor,cm_Ethane_conv_factor_sigma,cm_Methane_cradle-to-gate,cm_Methane_cradle-to-gate_sigma,cm_Methane_conv_factor,cm_Methane_conv_factor_sigma,cm_Nitric oxide_cradle-to-gate,cm_Nitric oxide_cradle-to-gate_sigma,cm_Nitric oxide_conv_factor,cm_Nitric oxide_conv_factor_sigma,cm_Nitrogen fluoride_cradle-to-gate,cm_Nitrogen fluoride_cradle-to-gate_sigma,cm_Nitrogen fluoride_conv_factor,cm_Nitrogen fluoride_conv_factor_sigma,cm_Perfluoropentane_cradle-to-gate,cm_Perfluoropentane_cradle-to-gate_sigma,cm_Perfluoropentane_conv_factor,cm_Perfluoropentane_conv_factor_sigma,cm_Sulfur hexafluoride_cradle-to-gate,cm_Sulfur hexafluoride_cradle-to-gate_sigma,cm_Sulfur hexafluoride_conv_factor,cm_Sulfur hexafluoride_conv_factor_sigma,cm_Other_cradle-to-gate,cm_Other_cradle-to-gate_sigma,cm_Other_conv_factor,cm_Other_conv_factor_sigma
3801,2M-195,2021 Q3,COOLING WATER,Direct Utilities,"1,12-DODECANEDIAMINE (DMDA) FROM 1,12-DODECANE...",1990.0,Germany,"1,12-DODECANEDIAMINE",178.474735,kg/kg,MM KG/yr,Utilities,IHS PEP,8.923737,COOLING WATER,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [50]:
# Write to file
output_emissions.to_csv(output_path+'ihsMaterialsEmissions_w_uncertainties.csv')

In [74]:
# Create match list for direct processes to IHS products

# product_list = input_emissions['Product'].sort_values().drop_duplicates()
# matches = direct_emissions.merge(product_list, left_on='Process', right_on='Product', how='left', suffixes=('','_match'))
# matches[['Product', 'Process']].to_csv(input_path+'extra_inputs/product_to_directProcess_matches.csv')